<a href="https://colab.research.google.com/github/crossrobot/2021-Bigcontest-SportTech-Sector-YonseiDSL-/blob/main/VAR_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
#module 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
import os
import random
import datetime
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#데이터 가져오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# VAR

## Data Load & Preprocessing

In [ ]:
path =  '/content/drive/Shareddrives/Big Contest - BaseBall/빅콘테스트-야구/Final_Data/선수_경기별_데이터/'
강백호_경기별 = pd.read_csv(os.path.join(path, '강백호경기별데이터.csv'), encoding = 'CP949')
로맥_경기별 = pd.read_csv(os.path.join(path, '로맥경기별데이터.csv'), encoding = 'CP949')
최정_경기별 = pd.read_csv(os.path.join(path, '최정경기별데이터.csv'), encoding = 'CP949')
이정후_경기별 = pd.read_csv(os.path.join(path, '이정후경기별데이터.csv'), encoding = 'CP949')
채은성_경기별 = pd.read_csv(os.path.join(path, '채은성경기별데이터.csv'), encoding = 'CP949')
양의지_경기별 = pd.read_csv(os.path.join(path, '양의지경기별데이터.csv'), encoding = 'CP949')
박건우_경기별 = pd.read_csv(os.path.join(path, '박건우경기별데이터.csv'), encoding = 'CP949')
김현수_경기별 = pd.read_csv(os.path.join(path, '김현수경기별데이터.csv'), encoding = 'CP949')
김재환_경기별 = pd.read_csv(os.path.join(path, '김재환경기별데이터.csv'), encoding = 'CP949')
전준우_경기별 = pd.read_csv(os.path.join(path, '전준우경기별데이터.csv'), encoding = 'CP949')

In [ ]:
# 날짜 정보를 생성하는 함수 
def date_generator(dataset) : 
  dataset['date'] = dataset.apply(lambda x: str(x['연도'])+'년 '+x['날짜'], axis=1)
  dataset['date'] = dataset['date'].apply(lambda x : datetime.datetime.strptime(x, '%Y년 %m월 %d일').date())
  dataset = dataset.set_index('date', drop = False)
  return dataset

In [ ]:
# 'date' 열 생성
강백호_경기별 = date_generator(강백호_경기별)
로맥_경기별 = date_generator(로맥_경기별)
최정_경기별 = date_generator(최정_경기별)
이정후_경기별 = date_generator(이정후_경기별)
채은성_경기별 = date_generator(채은성_경기별)
양의지_경기별 = date_generator(양의지_경기별)
박건우_경기별 = date_generator(박건우_경기별)
김현수_경기별 = date_generator(김현수_경기별)
김재환_경기별 = date_generator(김재환_경기별)
전준우_경기별 = date_generator(전준우_경기별)

In [ ]:
# 정규시즌 데이터 필터링 함수
def season_selector(df) : 
  startdate2021 = datetime.datetime.strptime('2021-04-03', "%Y-%m-%d").date()
  enddate2021 = datetime.datetime.strptime('2021-10-08', "%Y-%m-%d").date()
  startdate2020 = datetime.datetime.strptime('2020-05-05', "%Y-%m-%d").date()
  enddate2020 = datetime.datetime.strptime('2020-10-31', "%Y-%m-%d").date()
  startdate2019 = datetime.datetime.strptime('2019-03-23', "%Y-%m-%d").date()
  enddate2019 = datetime.datetime.strptime('2019-10-01', "%Y-%m-%d").date()
  startdate2018 = datetime.datetime.strptime('2018-03-24', "%Y-%m-%d").date()
  enddate2018 = datetime.datetime.strptime('2018-09-30', "%Y-%m-%d").date()

  filtered = df[(((df['date'] >= startdate2021) & (df['date'] <= enddate2021)) | ((df['date'] >= startdate2020) & (df['date'] <=  enddate2020)) | ((df['date'] >= startdate2019) & (df['date'] <= enddate2019)) | ((df['date'] >= startdate2018) & (df['date'] <= enddate2018)) )]
  return filtered

In [ ]:
# 필터링 된 데이터 
강백호_경기별=season_selector(강백호_경기별)
로맥_경기별=season_selector(로맥_경기별)
최정_경기별=season_selector(최정_경기별)
이정후_경기별=season_selector(이정후_경기별)
채은성_경기별=season_selector(채은성_경기별)
양의지_경기별=season_selector(양의지_경기별)
박건우_경기별=season_selector(박건우_경기별)
김현수_경기별=season_selector(김현수_경기별)
김재환_경기별=season_selector(김재환_경기별)
전준우_경기별=season_selector(전준우_경기별)

In [ ]:
# 아웃라이어 필터링 함수  #why
def outlier_filter(df) : 
  filtered_df = df[(df['OPS']>=0.75) & (df['OPS']<=1.2)]
  return filtered_df   

In [ ]:
# 필터링 된 데이터 
강백호_경기별=outlier_filter(강백호_경기별)
로맥_경기별=outlier_filter(로맥_경기별)
최정_경기별=outlier_filter(최정_경기별)
이정후_경기별=outlier_filter(이정후_경기별)
채은성_경기별=outlier_filter(채은성_경기별)
양의지_경기별=outlier_filter(양의지_경기별)
박건우_경기별=outlier_filter(박건우_경기별)
김현수_경기별=outlier_filter(김현수_경기별)
김재환_경기별=outlier_filter(김재환_경기별)
전준우_경기별=outlier_filter(전준우_경기별)

In [ ]:
# 타구와 관계가 없다고 판단되는  열들은 제거
강백호_경기별 = 강백호_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구', '희타', '희비','date' ] , axis=1)
로맥_경기별 = 로맥_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구', '희타', '희비','date'] , axis=1)
최정_경기별 = 최정_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구','희타', '희비','date' ] , axis=1)    
이정후_경기별 = 이정후_경기별.drop(['연도','날짜','상대','선발','결과','타순','P','투구','희타', '희비','date'] , axis=1)
채은성_경기별 = 채은성_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구','희타', '희비' ,'date'] , axis=1)
양의지_경기별 = 양의지_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구','희타', '희비' ,'date'] , axis=1)
박건우_경기별 = 박건우_경기별.drop(['연도','날짜','상대','선발','결과','타순','P',  '투구','희타', '희비','date'] , axis=1)
김현수_경기별 = 김현수_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구','희타', '희비','date' ] , axis=1)
김재환_경기별 = 김재환_경기별.drop(['연도','날짜','상대','선발','결과','타순','P',  '투구' ,'희타', '희비','date'] , axis=1)
전준우_경기별 = 전준우_경기별.drop(['연도','날짜','상대','선발','결과','타순','P', '투구','희타', '희비','date' ] , axis=1)

선수_경기별_list = [강백호_경기별, 로맥_경기별, 최정_경기별, 이정후_경기별, 채은성_경기별, 양의지_경기별, 박건우_경기별, 김현수_경기별, 김재환_경기별, 전준우_경기별]

## Modeling

### 1. ADF test
정상성 검정을 통해 어떤 변수를 차분해야할지 결정한다.

각 선수별로
adf테스트 > 차분할 변수 선정 > 리스트에 담기 > 그 변수만 대상으로 adf테스트 > 그 변수를 포함하여 df 만들기 

In [ ]:
선수 = ['강백호', '로맥', '최정', '이정후', '채은성', '양의지', '박건우', '김현수', '김재환', '전준우']
for 선수_경기별, j in tqdm(zip(선수_경기별_list,선수)):
    print("---------------------------------------------------------------------------------------------------------")
    print(j)
    df = 선수_경기별.copy()
    candidate = []
    for i in 선수_경기별.columns[:-1]: 
       adfuller_test = adfuller(df[i], autolag= "AIC")
       p_value = adfuller_test[1]
       if p_value > 0.05:
         candidate.append(i) #차분 후보 변수 찾기
    print(candidate)
    candidate2 = []
    if candidate !=[]:
        for k in candidate:
            df2 = df[[k]]
            df2 = df2.diff().dropna()
            adfuller_test = adfuller(df2[k], autolag= "AIC")
            if p_value > 0.05:
                candidate2.append(k) #차분 후보 변수 찾기
    print(candidate2) 

0it [00:00, ?it/s]

---------------------------------------------------------------------------------------------------------
강백호


1it [00:00,  1.77it/s]

['타율']
[]
---------------------------------------------------------------------------------------------------------
로맥


2it [00:01,  1.48it/s]

...

['장타', 'OPS']
[]
---------------------------------------------------------------------------------------------------------
김재환


9it [00:04,  2.06it/s]

['장타']
[]
---------------------------------------------------------------------------------------------------------
전준우


10it [00:04,  2.04it/s]

['장타']
[]


ADF test결과, 1회 차분으로 모든 변수가 정상성을 가짐을 알 수 있다. 

In [ ]:
선수 = ['강백호', '로맥', '최정', '이정후', '채은성', '양의지', '박건우', '김현수', '김재환', '전준우']
final_mse_ops = []
final_mse_출루 = []
final_mse_장타 = []
for 선수_경기별, j in tqdm(zip(선수_경기별_list,선수)):
  print("---------------------------------------------------------------------------------------------------------")
  print(j)
  df = 선수_경기별.copy()
  candidate = []
  for i in 선수_경기별.columns: 
    adfuller_test = adfuller(df[i], autolag= "AIC")
    p_value = adfuller_test[1]
    if p_value > 0.05:
       candidate.append(i) #차분 후보 변수 만들기
  if candidate != []:
    df2 = df[candidate]
    df2 = df2.diff().fillna(0)
    df3 = df.drop(candidate, axis = 1)
    df4 = pd.concat([df2,df3],axis=1)
    train_df = df4[:-21].copy()
    test_df = df[-21:].copy()  
    forecasting_model = VAR(train_df) 
    results = forecasting_model.fit(5) #Grid Search 결과, lag=5가 최적이 나왔음. 
    lagged_values = train_df.values[-5:]
    forecast = pd.DataFrame(results.forecast(y =lagged_values, steps=21), index = test_df.index, columns = df4.columns)    
    for a in candidate:
        forecast[a] = df[a].iloc[-21-1] + forecast[a].cumsum()
        table = pd.merge(forecast[['OPS','출루','장타']], test_df[['OPS','출루','장타']],left_on = 'date', right_on = 'date')
    final_mse_ops.append(np.sqrt(mean_squared_error(table[['OPS_x']],table[['OPS_y']])))
    final_mse_출루.append(np.sqrt(mean_squared_error(table[['출루_x']],table[['출루_y']])))
    final_mse_장타.append(np.sqrt(mean_squared_error(table[['장타_x']],table[['장타_y']])))

0it [00:00, ?it/s]

---------------------------------------------------------------------------------------------------------
강백호


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
1it [00:00,  1.71it/s]

---------------------------------------------------------------------------------------------------------
로맥


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
2it [00:01,  1.51it/s]

...

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
8it [00:04,  2.04it/s]

---------------------------------------------------------------------------------------------------------
김재환


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
9it [00:04,  1.92it/s]

---------------------------------------------------------------------------------------------------------
전준우


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
10it [00:05,  1.92it/s]


In [ ]:
table

,OPS_x,출루_x,장타_x,OPS_y,출루_y,장타_y
date,,,,,,
2021-08-18,0.836724,0.400559,0.436260,0.817,0.395,0.422
2021-08-20,0.835613,0.397566,0.438355,0.811,0.392,0.419
2021-08-22,0.842650,0.400088,0.442942,0.802,0.388,0.414
2021-08-23,0.850491,0.403109,0.447736,0.805,0.390,0.414
2021-08-25,0.849596,0.402108,0.448049,0.807,0.390,0.417
2021-08-26,0.846436,0.400890,0.445865,0.808,0.391,0.416
2021-08-28,0.849053,0.401144,0.448296,0.813,0.392,0.422
2021-08-29,0.851328,0.401771,0.449922,0.813,0.390,0.423
2021-08-31,0.851081,0.400209,0.451268,0.811,0.389,0.422


In [ ]:
d = {'OPS':final_mse_ops,'출루':final_mse_출루, '장타':final_mse_장타}
df = pd.DataFrame(data=d, index = 선수)
df

,OPS,출루,장타
강백호,0.016663,0.021875,0.014944
로맥,0.014483,0.007441,0.011644
최정,0.043487,0.010180,0.034315
이정후,0.043244,0.016744,0.026299
채은성,0.021014,0.009778,0.018329
양의지,0.030862,0.009982,0.021034
박건우,0.030986,0.024457,0.018885
김현수,0.085358,0.015613,0.069765
김재환,0.021930,0.008403,0.015491
전준우,0.045642,0.012912,0.033242


In [ ]:
df['OPS'].mean()

0.03536674534681935

In [ ]:
df['출루'].mean()

0.013738461016551618

In [ ]:
df['장타'].mean()

0.02639475161035551

## Model Training

In [ ]:
선수 = ['강백호', '로맥', '최정', '이정후', '채은성', '양의지', '박건우', '김현수', '김재환', '전준우']
object_index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
final_forecast = []
for 선수_경기별, j in tqdm(zip(선수_경기별_list,선수)):
  print("---------------------------------------------------------------------------------------------------------")
  print(j)
  df = 선수_경기별.copy()
  candidate = []
  for i in 선수_경기별.columns: 
    adfuller_test = adfuller(df[i], autolag= "AIC")
    p_value = adfuller_test[1]
    if p_value > 0.05:
       candidate.append(i) #차분 후보 변수 만들기
  if candidate != []:
    df2 = df[candidate]
    df2 = df2.diff().fillna(0)
    df3 = df.drop(candidate, axis = 1)
    df4 = pd.concat([df2,df3],axis=1) #최종예측에 사용되는 data 
    forecasting_model = VAR(df4)
    results = forecasting_model.fit(5) #Grid Search 결과, lag=5가 최적이 나왔음. 
    lagged_values = df4.values[-5:]
    forecast = pd.DataFrame(results.forecast(y =lagged_values, steps=21), index = object_index , columns = df4.columns)    
    for a in candidate:
        forecast[a] = df[a].iloc[-21-1] + forecast[a].cumsum()
    print(" ")
    print(forecast[['OPS','장타','출루']])
    final = forecast[['OPS','장타','출루']].mean()
    final_forecast.append(final)
  
     

0it [00:00, ?it/s]

---------------------------------------------------------------------------------------------------------
강백호


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
1it [00:00,  1.72it/s]

 
         OPS        장타        출루
1   1.050209  0.592663  0.457480
2   1.014992  0.565357  0.449501
3   1.009093  0.559519  0.449831
4   1.006883  0.560433  0.446390
5   1.009408  0.563650  0.445764
6   1.010133  0.561393  0.448844
7   1.010584  0.563109  0.447469
8   0.999874  0.557249  0.442640
9   0.999783  0.556041  0.443693
10  0.996942  0.555411  0.441513
11  0.994842  0.554293  0.440551
12  0.995239  0.554652  0.440583
13  0.995358  0.555027  0.440364
14  0.995080  0.554948  0.440115
15  0.995124  0.555333  0.439782
16  0.995179  0.555564  0.439618
17  0.993163  0.554458  0.438692
18  0.992396  0.554170  0.438220
19  0.990865  0.553380  0.437481
20  0.990002  0.553017  0.436981
21  0.989462  0.552778  0.436681
---------------------------------------------------------------------------------------------------------
로맥


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
2it [00:01,  1.57it/s]

...

 
         OPS        장타        출루
1   0.859960  0.481331  0.381039
2   0.866617  0.486839  0.382086
3   0.872333  0.491529  0.383382
4   0.878712  0.496324  0.384692
5   0.889758  0.504933  0.387200
6   0.895105  0.508382  0.389050
7   0.900931  0.512567  0.390710
8   0.902836  0.513276  0.391930
9   0.904687  0.514473  0.392582
10  0.907460  0.516832  0.392995
11  0.910180  0.519231  0.393315
12  0.911537  0.520266  0.393647
13  0.913277  0.521471  0.394179
14  0.915998  0.523489  0.394907
15  0.917721  0.524860  0.395262
16  0.919904  0.526631  0.395670
17  0.922130  0.528464  0.396072
18  0.923770  0.529804  0.396371
19  0.925601  0.531274  0.396742
20  0.927143  0.532516  0.397050
21  0.928526  0.533658  0.397293
---------------------------------------------------------------------------------------------------------
김재환


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
9it [00:04,  1.90it/s]

 
         OPS        장타        출루
1   0.905211  0.501515  0.391316
2   0.895101  0.500343  0.383164
3   0.902795  0.504796  0.385774
4   0.883563  0.488764  0.382651
5   0.880479  0.492605  0.375940
6   0.889557  0.497262  0.380344
7   0.875252  0.485298  0.378122
8   0.867440  0.481112  0.374242
9   0.873305  0.487663  0.373586
10  0.875659  0.489633  0.374166
11  0.875439  0.489182  0.374323
12  0.874982  0.488604  0.374431
13  0.878168  0.490971  0.375221
14  0.879858  0.492046  0.375886
15  0.881523  0.493470  0.376139
16  0.882827  0.494410  0.376467
17  0.884943  0.495996  0.376998
18  0.885813  0.496483  0.377398
19  0.887317  0.497571  0.377815
20  0.888283  0.498204  0.378144
21  0.889632  0.499332  0.378354
---------------------------------------------------------------------------------------------------------
전준우


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
10it [00:05,  1.92it/s]

 
         OPS        장타        출루
1   0.811906  0.425550  0.382081
2   0.824210  0.434669  0.385135
3   0.833445  0.435494  0.393766
4   0.841132  0.443474  0.393851
5   0.823953  0.432906  0.387153
6   0.815921  0.431282  0.380670
7   0.823966  0.436993  0.382747
8   0.832794  0.442773  0.385889
9   0.838349  0.446916  0.387540
10  0.836469  0.446082  0.386364
11  0.837535  0.448464  0.385062
12  0.840635  0.450269  0.386240
13  0.843843  0.452845  0.386957
14  0.845272  0.454289  0.386974
15  0.845072  0.454811  0.386218
16  0.846913  0.456653  0.386198
17  0.848711  0.458186  0.386455
18  0.850878  0.460122  0.386719
19  0.852072  0.461367  0.386667
20  0.853103  0.462700  0.386359
21  0.854300  0.463980  0.386261


In [ ]:
final_forecast

[OPS    1.001648
 장타     0.558688
 출루     0.442962
 dtype: float64, OPS    0.811149
 장타     0.455432
 출루     0.349675
 dtype: float64, OPS    0.977100
 장타     0.580557
 출루     0.408333
 dtype: float64, OPS    0.955882
 장타     0.524956
 출루     0.432620
 dtype: float64, OPS    0.882201
 장타     0.511440
 출루     0.371045
 dtype: float64, OPS    1.090142
 장타     0.641968
 출루     0.425767
 dtype: float64, OPS    0.834658
 장타     0.445102
 출루     0.384811
 dtype: float64, OPS    0.904485
 장타     0.515150
 출루     0.391723
 dtype: float64, OPS    0.883674
 장타     0.493584
 출루     0.378118
 dtype: float64, OPS    0.838118
 장타     0.447611
 출루     0.386443
 dtype: float64]

In [ ]:
pd.DataFrame(final_forecast, index = 선수 ) #최종예측

,OPS,장타,출루
강백호,1.001648,0.558688,0.442962
로맥,0.811149,0.455432,0.349675
최정,0.977100,0.580557,0.408333
이정후,0.955882,0.524956,0.432620
채은성,0.882201,0.511440,0.371045
양의지,1.090142,0.641968,0.425767
박건우,0.834658,0.445102,0.384811
김현수,0.904485,0.515150,0.391723
김재환,0.883674,0.493584,0.378118
전준우,0.838118,0.447611,0.386443
